<a href="https://colab.research.google.com/github/musicjae/GNN/blob/main/2_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

원문: https://github.com/emergx/gnn-tutorials/blob/master/1_gcn.ipynb  
paper: https://arxiv.org/pdf/1609.02907.pdf

In [1]:
!pip install networkx
!pip install dgl

     |████████████████████████████████| 3.6MB 11.2MB/s 


- 메시지 패싱 관점에서 GCN  
 - 이웃 표현neighbor'srepresentations $h_v$를 중간 표현 $\hat{h_u}$를 만들기 위해 총합한다. 총합된 표현 $\hat{h_u}$을 선형 투영 뒤에 따라 나오는 비선형성인 $h_u = f(W_u\hat{h_u})$으로 변환해준다.
   


과정  
  
-  message 정의  
-  평범한 함수로 바꿔줌

# 1 임포트

In [18]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

In [8]:
gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

 A GCNLayer essentially performs message passing on all the nodes then applies a fully-connected layer.

In [12]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats) # Fcnet

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)
        

cora dataset (the input feature size is 1433 and the number of classes is 7

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)
    
    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [17]:
from dgl.data import citation_graph as citegrh
import networkx as nx

def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask


## 평가

In [19]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [20]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch >=3:
        dur.append(time.time() - t0)
    
    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

Loading from cache failed, re-processing.
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


/usr/local/lib/python3.6/dist-packages/dgl/data/utils.py:285: UserWarning: Property dataset.feat will be deprecated, please use g.ndata['feat'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.6/dist-packages/dgl/data/utils.py:285: UserWarning: Property dataset.label will be deprecated, please use g.ndata['label'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.6/dist-packages/dgl/data/utils.py:285: UserWarning: Property dataset.train_mask will be deprecated, please use g.ndata['train_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))
/usr/local/lib/python3.6/dist-packages/dgl/data/utils.py:285: UserWarning: Property dataset.test_mask will be deprecated, please use g.ndata['test_mask'] instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


Epoch 00000 | Loss 1.9595 | Test Acc 0.3740 | Time(s) nan
Epoch 00001 | Loss 1.8513 | Test Acc 0.4180 | Time(s) nan
Epoch 00002 | Loss 1.7336 | Test Acc 0.4840 | Time(s) nan
Epoch 00003 | Loss 1.6334 | Test Acc 0.5330 | Time(s) 0.0540
Epoch 00004 | Loss 1.5279 | Test Acc 0.5810 | Time(s) 0.0518
Epoch 00005 | Loss 1.4311 | Test Acc 0.6320 | Time(s) 0.0517
Epoch 00006 | Loss 1.3406 | Test Acc 0.6730 | Time(s) 0.0515
Epoch 00007 | Loss 1.2509 | Test Acc 0.7400 | Time(s) 0.0513
Epoch 00008 | Loss 1.1595 | Test Acc 0.7810 | Time(s) 0.0511
Epoch 00009 | Loss 1.0688 | Test Acc 0.7730 | Time(s) 0.0517
Epoch 00010 | Loss 0.9821 | Test Acc 0.7600 | Time(s) 0.0515
Epoch 00011 | Loss 0.9013 | Test Acc 0.7560 | Time(s) 0.0513
Epoch 00012 | Loss 0.8262 | Test Acc 0.7580 | Time(s) 0.0513
Epoch 00013 | Loss 0.7558 | Test Acc 0.7580 | Time(s) 0.0513
Epoch 00014 | Loss 0.6900 | Test Acc 0.7660 | Time(s) 0.0511
Epoch 00015 | Loss 0.6287 | Test Acc 0.7680 | Time(s) 0.0511
Epoch 00016 | Loss 0.5718 | Test 